### 写github的main.py调试代码

In [1]:
from model import *
import fit

from Mol2Graph import Mol2Graph
from rdkit.Chem import AllChem as Chem
from multiprocessing import Pool
import numpy as np

import torch
import random
from torch_geometric.data import Data #, DataLoader
from torch_geometric.loader import DataLoader
from multiprocessing import Pool

In [3]:
fr_Train = eval(open('../density/21_split/Train_exc_items.table').read())
fr_test = eval(open('../density/21_split/test_items.table').read())

In [4]:
class DataFeat(object):
    def __init__(self, **kwargs):
        for k in kwargs:
            self.__dict__[k] = kwargs[k]
            
mol_block = eval(open('../density/21_split/density_molblocks_2021.dir').read())
def make_data(items):
    try:
        mol1 = Chem.MolFromMolBlock(mol_block[items[0]])
        mol2 = Chem.MolFromMolBlock(mol_block[items[1]])

        g1 = Mol2Graph(mol1)
        g2 = Mol2Graph(mol2)
        x = np.concatenate([g1.x, g2.x], axis=0)
        edge_feats = np.concatenate([g1.edge_feats, g2.edge_feats], axis=0)
        e_idx2 = g2.edge_idx+g1.node_num
        edge_index = np.concatenate([g1.edge_idx, e_idx2], axis=0)
        return DataFeat(x=x, edge_feats=edge_feats, edge_index=edge_index, y=np.array([float(items[2])]))
    except: 
        print('Bad input sample:'+items[-1]+'skipped.')

In [8]:
"""
items = ['S1C=CSC1=C1SC=CS1_TTFCAN15', 'ClC1=C(Cl)C(=O)C(=C(Cl)C1=O)Cl_TTFCAN15', 1.944, 'TTFCAN15']
mol1 = Chem.MolFromMolBlock(mol_block[items[0]])
mol2 = Chem.MolFromMolBlock(mol_block[items[1]])

g1 = Mol2Graph(mol1)
g2 = Mol2Graph(mol2)
x = np.concatenate([g1.x, g2.x], axis=0)
edge_feats = np.concatenate([g1.edge_feats, g2.edge_feats], axis=0)
e_idx2 = g2.edge_idx+g1.node_num
edge_index = np.concatenate([g1.edge_idx, e_idx2], axis=0)
"""

In [5]:
from multiprocessing import Pool

pool = Pool(processes=30)                 # 创建进程池对象，进程数与multiprocessing.cpu_count()相同
data_pool_Train = pool.map(make_data, [i for i in fr_Train if Chem.MolFromMolBlock(mol_block[i[0]]) != None and Chem.MolFromMolBlock(mol_block[i[1]]) != None])  # 并行执行函数
data_pool_test = pool.map(make_data, [i for i in fr_test if Chem.MolFromMolBlock(mol_block[i[0]]) != None and Chem.MolFromMolBlock(mol_block[i[1]]) != None])
pool.close()
pool.join()

In [6]:
Y_Train = np.array([i.y for i in data_pool_Train])
std = Y_Train.std()
mean = Y_Train.mean()
print('Mean:', mean, '\tStd:', std)

Mean: 1.4773111506223011 	Std: 0.2657192556909069


In [7]:
loader_Train = []
for d in data_pool_Train:
    i = Data(x=torch.tensor(d.x),   #使用PyG 的Data来建图
         edge_index=torch.tensor(d.edge_index.T, dtype=torch.long),
         edge_attr=torch.tensor(d.edge_feats),
         y=torch.tensor((d.y-mean)/std, dtype=torch.float32))
    loader_Train.append(i)

In [8]:
loader_test = []
for d in data_pool_test:
    i = Data(x=torch.tensor(d.x),  #使用PyG 的Data来建图
         edge_index=torch.tensor(d.edge_index.T, dtype=torch.long),
         edge_attr=torch.tensor(d.edge_feats),
         y=torch.tensor((d.y-mean)/std, dtype=torch.float32))
    loader_test.append(i)

In [9]:
random.seed(1000)
random.shuffle(loader_Train)

train_loader = DataLoader(loader_Train, batch_size=128, shuffle=0, drop_last=True)
test_loader = DataLoader(loader_test, batch_size=128, shuffle=0)

In [10]:
model_name = 'test_github'
data_loaders = {'train':train_loader, 'valid':test_loader}
fit.training(CCPGraph, data_loaders, n_epoch=100, save_att=True, snapshot_path='./snapshot_Bayes_Opt/{}//'.format(model_name), mean=mean, std=std)

/PUBLIC/home/guojiali/anaconda3/envs/ccdc/lib/python3.7/site-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch:001 ==> LR: 0.001000, Train loss: 0.0172, Train R2_pccs: 0.8614, Train_R2: 0.7530, Valid loss: 0.0168, Valid R2_pccs: 0.8959, Valid R2: 0.7753, Elapsed Time:9.64 s


/PUBLIC/home/guojiali/anaconda3/envs/ccdc/lib/python3.7/site-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch:002 ==> LR: 0.001000, Train loss: 0.0074, Train R2_pccs: 0.9255, Train_R2: 0.8934, Valid loss: 0.0075, Valid R2_pccs: 0.9557, Valid R2: 0.8997, Elapsed Time:1.59 s


/PUBLIC/home/guojiali/anaconda3/envs/ccdc/lib/python3.7/site-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


...

/PUBLIC/home/guojiali/anaconda3/envs/ccdc/lib/python3.7/site-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch:099 ==> LR: 0.000216, Train loss: 0.0051, Train R2_pccs: 0.9282, Train_R2: 0.9267, Valid loss: 0.0019, Valid R2_pccs: 0.9766, Valid R2: 0.9746, Elapsed Time:1.14 s


/PUBLIC/home/guojiali/anaconda3/envs/ccdc/lib/python3.7/site-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch:100 ==> LR: 0.000216, Train loss: 0.0050, Train R2_pccs: 0.9294, Train_R2: 0.9279, Valid loss: 0.0021, Valid R2_pccs: 0.9791, Valid R2: 0.9726, Elapsed Time:1.19 s

Loss: 0.0016123481029599617


{'valid mae': 0.03066898428875467,
 'valid loss': 0.0016123481029599617,
 'valid R2': 0.9784764451060166}